In [1]:
import numpy as np
from scipy.stats import mode
from pbil import pbil
import multiprocessing
%pylab inline
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


In [2]:
data = np.loadtxt('../data/ImageRawReduced.txt')
targets = np.loadtxt('../data/ImageExpertReduced.txt')
rules = np.loadtxt('../data/ClassificationRules.txt')
data.shape, targets.shape, rules.shape

((3, 9350), (9350,), (266, 9350))

In [3]:
def draw_plots(scores_history, props_history, problem_title=None):
    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(15,4))
    if problem_title:
        fig.suptitle(problem_title, fontsize=20)
    ax1.plot(np.arange(len(scores_history)), scores_history)
    ax1.set_xlabel('Iteration')
    ax1.set_ylabel('Score')
    ax1.set_title('Progress')

    
    for x in np.array(props_history).T:
        ax2.plot(np.arange(x.size), x)
        ax2.set_title('Propabilities')
        ax2.set_xlabel('Iteration')
        ax2.set_ylabel('Propability')
    plt.show()

In [4]:
def process_instance(rules_mask):
            return (mode(
                rules[rules_mask],
                axis=0
            )[0] == targets).sum()
    
def f(pop):
    res = np.zeros((pop.shape[0], targets.size))
    with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
        res = pool.map(process_instance, [x for x in pop])
    return np.array(res)

In [ ]:
%%time
res = pbil(
    f,
    vec_size=rules.shape[0],
    pop_size=100,
    iterations=4000,
    theta1=0.1,
    theta2=0.07,
    theta3=0.05,
)
draw_plots(res[1], res[2], 'Spectral')